In [12]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import keras

import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


from keras.layers import Dense,Dropout,LeakyReLU
from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt


def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [13]:
train = pd.read_csv('train.csv')
print('Shape of the train data with all features:', train.shape)
train = train.select_dtypes(exclude=['object'])
print("")
print('Shape of the train data with numerical features:', train.shape)
train.drop('Id',axis = 1, inplace = True)
train.fillna(0,inplace=True)

test = pd.read_csv('test.csv')
test = test.select_dtypes(exclude=['object'])
ID = test.Id
test.fillna(0,inplace=True)
test.drop('Id',axis = 1, inplace = True)

print("")
print("List of features contained our dataset:",list(train.columns))



Shape of the train data with all features: (1460, 81)

Shape of the train data with numerical features: (1460, 38)

List of features contained our dataset: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


In [14]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])


train.head(10)


import warnings
warnings.filterwarnings('ignore')

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('SalePrice')

mat_train = np.matrix(train)
mat_test  = np.matrix(test)
mat_new = np.matrix(train.drop('SalePrice',axis = 1))
mat_y = np.array(train.SalePrice).reshape((1314,1))

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
test  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis)

train.head()




# List of features
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "SalePrice"

# Columns
feature_cols = FEATURES

# Training set and Prediction set with the features to predict
training_set = train[COLUMNS]
prediction_set = train.SalePrice

# Train and Test
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES] , prediction_set, test_size=0.33, random_state=42)
y_train = pd.DataFrame(y_train, columns = [LABEL])
training_set = pd.DataFrame(x_train, columns = FEATURES).merge(y_train, left_index = True, right_index = True)
training_set.head()

# Training for submission
training_sub = training_set[col_train]



# Same thing but for the test set
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)
testing_set.head()

C:\Users\biswa\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
C:\Users\biswa\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


Number of Outliers: 146
Number of rows without outliers: 1314


C:\Users\biswa\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
1232,0.235294,0.239617,0.074221,0.625,0.571429,0.914729,0.800000,0.00000,0.574882,0.0,...,0.343750,0.467033,0.0,0.0,0.0,0.0,0.000000,0.454545,0.00,0.556191
76,0.352941,0.000000,0.063505,0.625,0.428571,0.891473,0.766667,0.01375,0.000000,0.0,...,0.163043,0.197802,0.0,0.0,0.0,0.0,0.045161,0.363636,0.75,0.309966
962,0.176471,0.159744,0.055205,0.375,0.142857,0.527132,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.636364,0.00,0.130767
433,0.588235,0.102236,0.028107,0.500,0.428571,0.914729,0.800000,0.07250,0.528892,0.0,...,0.000000,0.343407,0.0,0.0,0.0,0.0,0.000000,0.363636,0.00,0.299493
1110,0.823529,0.115016,0.011665,0.625,0.428571,0.953488,0.883333,0.06625,0.000000,0.0,...,0.000000,0.074176,0.0,0.0,0.0,0.0,0.000000,0.454545,1.00,0.326257


In [15]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor


seed = 7
np.random.seed(seed)


# def Leaky(x):
#     alpha=0.3
#     if x<0:
#         return alpha * x
#     return  x



Leaky = lambda x: keras.activations.relu(x, alpha=0.05)

In [16]:
# Model
model = Sequential()
BatchNormalization()
model.add(Dense(128, input_dim=36, kernel_initializer='normal', activation=Leaky))
BatchNormalization()
Dropout(0.5)
model.add(Dense(256, kernel_initializer='normal', activation=Leaky))
BatchNormalization()
Dropout(0.5)
model.add(Dense(128, kernel_initializer='normal', activation=Leaky))
BatchNormalization()
Dropout(0.5)
model.add(Dense(1, kernel_initializer='normal'))

# Compile model
model.compile(metrics=['mean_squared_error'],loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())

feature_cols = training_set[FEATURES]
labels = training_set[LABEL].values

model.fit(np.array(feature_cols), np.array(labels), epochs=50, batch_size=10,verbose=2)



# Evaluation on the test set created by train_test_split
model.evaluate(np.array(feature_cols), np.array(labels))

model.summary()

Epoch 1/50
 - 0s - loss: 0.0120 - mean_squared_error: 0.0120
Epoch 2/50
 - 0s - loss: 0.0054 - mean_squared_error: 0.0054
Epoch 3/50
 - 0s - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 4/50
 - 0s - loss: 0.0042 - mean_squared_error: 0.0042
Epoch 5/50
 - 0s - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 6/50
 - 0s - loss: 0.0041 - mean_squared_error: 0.0041
Epoch 7/50
 - 0s - loss: 0.0040 - mean_squared_error: 0.0040
Epoch 8/50
 - 0s - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 9/50
 - 0s - loss: 0.0037 - mean_squared_error: 0.0037
Epoch 10/50
 - 0s - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 11/50
 - 0s - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 12/50
 - 0s - loss: 0.0037 - mean_squared_error: 0.0037
Epoch 13/50
 - 0s - loss: 0.0026 - mean_squared_error: 0.0026
Epoch 14/50
 - 0s - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 15/50
 - 0s - loss: 0.0037 - mean_squared_error: 0.0037
Epoch 16/50
 - 0s - loss: 0.0027 - mean_squared_error: 0.0027
Epoch 17/50
 - 0s

In [17]:
preds = model.predict(np.array(x_test))
print(preds.shape)
print(x_test.shape)
print(y_test.shape)


print(x_test.mean())
print(preds.mean())
print(rmse(preds[0],y_test))

# Predictions
feature_cols_test = testing_set[FEATURES]
labels_test = testing_set[LABEL].values

y = model.predict(np.array(feature_cols_test))
predictions = list(itertools.islice(y, testing_set.shape[0]))



predictions = prepro_y.inverse_transform(np.array(predictions).reshape(434,1))

reality = pd.DataFrame(prepro.inverse_transform(testing_set), columns = [COLUMNS]).SalePrice


(434, 1)
(434, 36)
(434, 1)
MSSubClass       0.209948
LotFrontage      0.184398
LotArea          0.074624
OverallQual      0.517569
OverallCond      0.511850
YearBuilt        0.708213
YearRemodAdd     0.585829
MasVnrArea       0.065625
BsmtFinSF1       0.260308
BsmtFinSF2       0.035377
BsmtUnfSF        0.264044
TotalBsmtSF      0.328654
1stFlrSF         0.340967
2ndFlrSF         0.204986
LowQualFinSF     0.003203
GrLivArea        0.378513
BsmtFullBath     0.145929
BsmtHalfBath     0.021889
FullBath         0.507680
HalfBath         0.185484
BedroomAbvGr     0.472734
KitchenAbvGr     0.023041
TotRmsAbvGrd     0.379160
Fireplaces       0.209677
GarageYrBlt      0.963448
GarageCars       0.450461
GarageArea       0.344681
WoodDeckSF       0.126140
OpenPorchSF      0.104769
EnclosedPorch    0.062526
3SsnPorch        0.006178
ScreenPorch      0.037471
PoolArea         0.001798
MiscVal          0.003609
MoSold           0.478425
YrSold           0.457373
dtype: float64
0.35721633
SalePrice 

In [18]:
y_predict = model.predict(np.array(test))

def to_submit(pred_y,name_out):
    y_predict = list(itertools.islice(pred_y, test.shape[0]))
    y_predict = pd.DataFrame(prepro_y.inverse_transform(np.array(y_predict).reshape(len(y_predict),1)), columns = ['SalePrice'])
    y_predict = y_predict.join(ID)
    y_predict.to_csv(name_out + '.csv',index=False)

to_submit(y_predict, "submission_31")